## Introductie

Met deze notebook bouw je een boezemmodel voor Nooderzijlvest.

Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/openearth/delft3dfmpy#installation">D-HYDAMO omgeving</a>. Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

De verwijzing naar bestanden is geschreven in het Nederlands. De rest van de code is geschreven in het Engels. Elk code-blok is voorzien van uitleg boven het desbetreffende blok, geschreven in het Nederlands.

Download de bestanden van <a href="https://drive.google.com/drive/folders/1BXNEwlEDCtShDj5KVuHQcukup-QLTuzR">Google Drive</a> en zet deze in de folder .\beheerregister

## Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.



In [1]:
from pathlib import Path

beheerregister = Path(r".\beheerregister").absolute().resolve()

invoerbestanden = {"modelgebied": "Bemalings_en_lozingsgebieden.shp",
                   "branches": "Afaanvoervakken_DHYDRO_v5.shp",
                   "profielpunten": "profielpunten_3feb2021_actueel.shp",
                   "bruggen": "Bruggen_(doorstroomopeningen).shp",
                   "duikers": "duikers.shp",
                   "sifons": "sifons.shp",
                   "stuwen": "stuwen.shp",
                   "inlaten": "inlaten.shp",
                   "gemalen": "gemalen_v2.shp",
                   "peilgebieden": "Peilgebieden.shp"}

for key, item in invoerbestanden.items():
    if not beheerregister.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister_pad.joinpath(item)}")

## Inlezen beheerregister in HyDAMO

### Aanmaken HyDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [2]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hydrotools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

hydamo = HyDAMO(extent_file=str(beheerregister.joinpath(invoerbestanden["modelgebied"])))

### Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (4 = manning) wordt toegekend
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [3]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["branches"]),
                           "branches",
                           attribute_filter={"HydroModel": "Boezemmodel_v4"},
                           column_mapping={"OVKIDENT": "code"},
                           )

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.branches["ruwheidstypecode"] = 4

hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

### Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* toekennen ruwheid (manning = 35)
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid

ToDo:
* verbeteren toekenning ruwheid

In [4]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch_1": ["WL", "Z1"], #Hier staan alleen Z1 en Z2 in
                                             "PRO_TYPE": "PRO"},
                           column_mapping={"PROIDENT": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH_1": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )

grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    prof_gdf = prof_gdf.sort_values("order")  #Staan ook punten met zelfde order
    first_point = prof_gdf.iloc[0]["geometry"]
    cum_dist = -1
    line = []
    for idx, (_, row) in enumerate(prof_gdf.iterrows()):
        geom = row["geometry"]
        distance = geom.distance(first_point)
        if distance > cum_dist:
            cum_dist = distance
            line += [(geom.x, geom.y, row["z"])]

    profiles[code] = [code, LineString(line)]

profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 4
profiles_gdf["ruwheidswaarde"] = 35
profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=True,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen geparameteriseerde profielen

Per brug moet er een profiel worden toegevoegd

In [5]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "parametrised_profiles",
                           column_mapping={"bodemh_bov": "bodemhoogtebovenstrooms",
                                           "bodemh_ben": "bodemhoogtebenedenstrooms",
                                           "doorstroom": "bodembreedte",
                                           "hoogte_ond": "hoogteinsteeklinkerzijde",
                                           })

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "bodemhoogtebovenstrooms": [frame["bodemhoogtebovenstrooms"].values[0]
                                    for code, frame in grouper],
        "bodemhoogtebenedenstrooms": [frame["bodemhoogtebenedenstrooms"].values[0]
                                      for code, frame in grouper],
        "bodembreedte": [frame["bodembreedte"].values[0]
                         for code, frame in grouper],
        "hoogteinsteeklinkerzijde": [frame["hoogteinsteeklinkerzijde"].values[0]
                                     for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }

gdf = gpd.GeoDataFrame(data)
gdf["geometry"] = gdf.apply((lambda x: LineString([[x["geometry"].x,
                                                    x["geometry"].y],
                                                  [x["geometry"].x+1,
                                                   x["geometry"].y+1]])),
                            axis=1)

gdf["codegerelateerdobject"] = gdf["code"].copy()
gdf["code"] = [f"PRO_{code}" for code in gdf["code"]]
gdf["hoogteinsteekrechterzijde"] = gdf["hoogteinsteeklinkerzijde"]
gdf["ruwheidswaarde"] = 15
gdf["ruwheidstypecode"] = 4
gdf["taludhellinglinkerzijde"] = 1
gdf["taludhellingrechterzijde"] = 1

hydamo.parametrised_profiles.set_data(gdf,
                                      index_col="code",
                                      check_columns=True,
                                      check_geotype=False)

hydamo.parametrised_profiles.snap_to_branch(hydamo.branches,
                                            snap_method="overal",
                                            maxdist=5)

hydamo.parametrised_profiles.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen bruggen
Per brug moet er een profiel worden toegevoegd

In [6]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "bridges",
                           column_mapping={"doorstro_2": "lengte",
                                           "bodemh_bov": "hoogteonderzijde"})

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "hoogteonderzijde": [frame["hoogteonderzijde"].values[0] for code,
                             frame in grouper],
        "lengte": [frame["lengte"].values[0] for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }
gdf = gpd.GeoDataFrame(data)

gdf["hoogtebovenzijde"] = gdf["hoogteonderzijde"] + 1

gdf["dwarsprofielcode"] = gdf["code"]
gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidstypecode"] = 4
gdf["ruwheidswaarde"] = 70

hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)
hydamo.bridges.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.bridges.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen duikers en siphons
duikers en sifons worden als cuverts aan het HyDAMO object toegekend

ToDo
* vorm moet naar HyDAMO codering worden omgeschreven

In [7]:
culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    column_mapping={"hoogte_a1": "hoogteopening",
                                                    "breedte": "breedteopening",
                                                    "bok_benede": "hoogtebinnenonderkantbenedenstrooms",
                                                    "bok_boven": "hoogtebinnenonderkantbovenstrooms",
                                                    "vorm": "vormcode"})

siphons_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   column_mapping={"hoogte": "hoogteopening",
                                                   "breedte": "breedteopening",
                                                   "bok_benede":"hoogtebinnenonderkantbenedenstrooms",
                                                   "bok_boven":"hoogtebinnenonderkantbovenstrooms",
                                                   "vorm":"vormcode"})

gdf = gpd.GeoDataFrame(pd.concat([culverts_gdf,siphons_gdf], ignore_index=True))

gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidswaarde"] = 70
gdf["ruwheidstypecode"] = 4

hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.culverts.snap_to_branch(hydamo.branches, snap_method="ends", maxdist=5)
hydamo.culverts.dropna(axis=0, inplace=True, subset=["branch_offset"])

hydamo.culverts.loc[hydamo.culverts["lengte"].isna(), "lengte"] = 20

### Toevoegen stuwen

In [8]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["stuwen"]),
                           "weirs",
                           column_mapping={"kstident": "code",
                                           "kstsoort": "soortstuwcode",
                                           "kstkrubr": "laagstedoorstroombreedte",
                                           "kstmikho": "laagstedoorstroomhoogte",
                                           "kstregel": "soortregelbaarheidcode"})

gdf["afvoercoefficient"] = 1

hydamo.weirs.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.weirs.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=10)
hydamo.weirs.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen gemalen

In [9]:
pumps_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["gemalen"]),
                                 "pumps",
                                 column_mapping={"gecom_max_": "maximalecapaciteit"})

gemalen_gdf = pumps_gdf.copy()
pumps_gdf["codegerelateerdobject"] = pumps_gdf["code"].copy()
pumps_gdf["code"] = [f"PMP_{code}" for code in pumps_gdf["code"]]
hydamo.gemalen.set_data(gemalen_gdf,
                        index_col="code",
                        check_columns=True,
                        check_geotype=True)
hydamo.gemalen.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.gemalen.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.pumps.set_data(pumps_gdf,
                      index_col="code",
                      check_columns=True,
                      check_geotype=True)
hydamo.pumps.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.pumps.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen afsluitmiddelen

In [10]:
gdf = pd.DataFrame({"code": [],
                    "soortafsluitmiddelcode": [],
                    "codegerelateerdobject": []})

hydamo.afsluitmiddel.set_data(gdf, index_col="code")

### Toevoegen sturing

In [11]:
gpg_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]),
                                 "sturing",
                                 column_mapping={"GPGIDENT": "code",
                                                 "OPVAFWZP": "streefwaarde"
                                                 })

gpg_gdf["codegerelateerdobject"] = [code.replace("GPG", "") for code in gpg_gdf["code"]]
gpg_gdf = gpg_gdf[gpg_gdf["codegerelateerdobject"].isin(hydamo.gemalen["code"])]
con_gdf = hydamo.gemalen[~hydamo.gemalen["code"].isin(gpg_gdf["codegerelateerdobject"])]
con_gdf = gpd.GeoDataFrame(data={"code": con_gdf["code"].values,
                                 "geometry": con_gdf["geometry"].values})
con_gdf["codegerelateerdobject"] = con_gdf["code"]
con_gdf["streefwaarde"] = -999

gdf = gpd.GeoDataFrame(pd.concat([gpg_gdf, con_gdf], ignore_index=True))

gdf["bovenmarge"] = gdf["streefwaarde"] + 0.05
gdf["ondermarge"] = gdf["streefwaarde"] - 0.05
gdf["doelvariabelecode"] = 1

hydamo.sturing.set_data(gdf, index_col="code")

## Converteren naar DFM

### Aanmaken dfm-klasse

In [12]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [13]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

2021-02-11 17:32:23,960 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - INFO - Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.
Culvert KDU005578 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU001149 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU037418 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KST010445 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU036814 has an unknown shape: None. Applying a default profile (round - 40cm)
Culvert KDU004710 has an unknown shape: heul. Applying a default profile (round - 40cm)
Culvert KDU046061 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU006612 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005350 has an unknown shape: muil

Culvert KDU005914 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005917 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005919 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005920 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005924 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005925 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005927 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005883 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005838 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005793 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005795 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU005799 has an unknown

Culvert KDU001645 has an unknown shape: heul. Applying a default profile (round - 40cm)
Culvert KDU001254 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU001266 has an unknown shape: ellips. Applying a default profile (round - 40cm)
Culvert KDU001598 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KDU001599 has an unknown shape: muil. Applying a default profile (round - 40cm)
Culvert KSY105 has an unknown shape: heul. Applying a default profile (round - 40cm)


### Aanmaken 1d netwerk

In [14]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=100.0, seperate_structures=True)

2021-02-11 17:32:27,857 - delft3dfmpy.core.dfm - dfm - WARNING - Some structures are not linked to a branch.
2021-02-11 17:32:27,869 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000010 at: [0.0, 691.94, 1642.4933902054406], due to the structures at [-0.001, 671.552, 712.328, 1163.135, 1539.892, 1642.4943902054406].
2021-02-11 17:32:27,872 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000015 at: [0.0, 106.271, 141.85887653227576], due to the structures at [-0.001, 78.108, 134.434, 141.85987653227576].
2021-02-11 17:32:27,876 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000017 at: [0.0, 506.58549999999997, 522.864, 971.4883654003871], due to the structures at [-0.001, 25.63, 498.27, 514.901, 530.827, 971.4893654003871].
2021-02-11 17:32:27,888 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000030 at: [0.0, 2.5425, 325.6167677365957], due to the structures at [-0.001, 0.1, 4.985, 325.61776

2021-02-11 17:32:28,163 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000603 at: [0.0, 199.723, 215.8915, 227.21174367409324], due to the structures at [-0.001, 191.286, 208.16, 223.623, 227.21274367409325].
2021-02-11 17:32:28,170 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000608 at: [0.0, 59.433499999999995, 130.5865, 163.78300000000002, 191.26850000000002, 1177.7512595417186], due to the structures at [-0.001, 8.422, 110.445, 150.728, 176.838, 205.699, 372.264, 1155.613, 1177.7522595417186].
2021-02-11 17:32:28,177 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000615 at: [0.0, 460.5815, 463.840805478718], due to the structures at [-0.001, 144.349, 322.782, 457.424, 463.739, 463.84180547871796].
2021-02-11 17:32:28,181 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000617 at: [0.0, 240.4535, 593.9086377149054], due to the structures at [-0.001, 217.858, 263.049, 390.768, 593.90963771

2021-02-11 17:32:28,390 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001032 at: [0.0, 13.7515, 41.93053779764144], due to the structures at [-0.001, 0.1, 27.403, 41.93153779764144].
2021-02-11 17:32:28,403 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001053 at: [0.0, 413.209, 867.2209302412621], due to the structures at [-0.001, 198.479, 307.814, 411.007, 415.411, 530.907, 657.645, 860.07, 867.221930241262].
2021-02-11 17:32:28,407 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001055 at: [0.0, 3.553, 1029.44431612395], due to the structures at [-0.001, 0.1, 7.006, 1029.44531612395].
2021-02-11 17:32:28,417 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001068 at: [0.0, 159.885, 186.759, 395.3107494999432], due to the structures at [-0.001, 72.58, 135.761, 184.009, 189.509, 388.321, 395.3117494999432].
2021-02-11 17:32:28,424 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on b

2021-02-11 17:32:28,697 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001493 at: [0.0, 827.081, 843.7500297007324], due to the structures at [-0.001, 292.981, 823.096, 831.066, 843.7510297007324].
2021-02-11 17:32:28,702 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001496 at: [0.0, 3.1224999999999996, 499.80837731493375], due to the structures at [-0.001, 0.1, 6.145, 155.643, 273.304, 499.7083773149337, 499.8093773149337].
2021-02-11 17:32:28,711 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001526 at: [0.0, 114.48023462832755, 148.0854692566551], due to the structures at [-0.001, 80.975, 147.9854692566551, 148.0864692566551].
2021-02-11 17:32:28,719 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001548 at: [0.0, 252.9325, 400.3128795319389], due to the structures at [-0.001, 12.071, 211.687, 294.178, 400.3138795319389].
2021-02-11 17:32:28,730 - delft3dfmpy.core.dfm - dfm - INFO - Added

2021-02-11 17:32:29,002 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002052 at: [0.0, 794.544, 816.8498072456608], due to the structures at [-0.001, 4.466, 283.299, 791.558, 797.53, 816.8508072456608].
2021-02-11 17:32:29,009 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KDU022690, KDU012505 have the same location.
2021-02-11 17:32:29,018 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002060 at: [0.0, 573.034, 604.505, 637.463, 648.1133047223738], due to the structures at [-0.001, 349.439, 570.499, 575.569, 633.441, 641.485, 648.1143047223737].
2021-02-11 17:32:29,027 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KDU006503, KDU006567 have the same location.
2021-02-11 17:32:29,046 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002076 at: [0.0, 172.48399999999998, 242.0425, 311.59000000000003, 311.59900000000005, 311.631, 735.9675, 735.98, 918.7170000000001, 918.7485, 918.78, 1189.4255, 1189.4475000000

...

2021-02-11 17:32:30,701 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005171 at: [0.0, 26.928, 50.931, 163.16780104897566], due to the structures at [-0.001, 6.897, 46.959, 54.903, 163.16880104897567].
2021-02-11 17:32:30,707 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005181 at: [0.0, 16.0745, 234.93076821167512], due to the structures at [-0.001, 0.1, 32.049, 228.838, 234.93176821167512].
2021-02-11 17:32:30,722 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005185 at: [0.0, 28.313, 52.512, 55.557, 60.368, 63.723, 100.1565, 169.978, 206.733, 212.3755, 215.337, 254.00631793341142, 292.75563586682284], due to the structures at [-0.001, 5.037, 51.589, 53.435, 57.679, 63.057, 64.389, 135.924, 204.032, 209.434, 215.317, 215.357, 292.6556358668228, 292.7566358668228].
2021-02-11 17:32:30,729 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005189 at: [0.0, 24.845000000000002, 52.0125, 93.6735,

2021-02-11 17:32:30,940 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005630 at: [0.0, 259.0295, 301.2105, 364.733, 434.93801526575993], due to the structures at [-0.001, 111.454, 226.092, 291.967, 310.454, 419.012, 434.9390152657599].
2021-02-11 17:32:30,943 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005631 at: [0.0, 1.8625, 55.965, 111.87322948526182], due to the structures at [-0.001, 0.1, 3.625, 108.305, 111.87422948526182].
2021-02-11 17:32:30,950 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005640 at: [0.0, 30.666999999999998, 196.1905, 228.84179309893926], due to the structures at [-0.001, 0.282, 61.052, 165.659, 226.722, 228.84279309893927].
2021-02-11 17:32:30,962 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005651 at: [0.0, 94.32650000000001, 132.2425, 200.55, 264.051, 299.21799999999996, 537.5730000000001, 663.0164518850748], due to the structures at [-0.001, 91.28, 97.37

2021-02-11 17:32:31,185 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006015 at: [0.0, 9.837000000000002, 30.1541729057869], due to the structures at [-0.001, 0.1, 19.574, 30.155172905786902].
2021-02-11 17:32:31,190 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006020 at: [0.0, 4.1305, 18.2575, 204.107, 226.5259525664145, 235.39190513282898], due to the structures at [-0.001, 0.1, 8.161, 28.354, 190.454, 217.76, 235.291905132829, 235.392905132829].
2021-02-11 17:32:31,199 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006025 at: [0.0, 32.271, 890.068272738783], due to the structures at [-0.001, 27.062, 37.48, 881.01, 890.069272738783].
2021-02-11 17:32:31,209 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006030 at: [0.0, 493.29758366756664, 541.5491673351332], due to the structures at [-0.001, 240.541, 445.146, 541.4491673351332, 541.5501673351332].
2021-02-11 17:32:31,221 - delft3dfmpy.

2021-02-11 17:32:31,477 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010227 at: [0.0, 988.03, 1026.9051278620673], due to the structures at [-0.001, 982.806, 993.254, 1026.9061278620673].
2021-02-11 17:32:31,484 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010231 at: [0.0, 88.72399999999999, 243.0647640314304], due to the structures at [-0.001, 56.854, 120.594, 243.0657640314304].
2021-02-11 17:32:31,488 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010232 at: [0.0, 55.614999999999995, 72.5838242426095], due to the structures at [-0.001, 47.698, 63.532, 72.5848242426095].
2021-02-11 17:32:31,492 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010234 at: [0.0, 59.7375, 141.43, 173.05937238907237], due to the structures at [-0.001, 0.1, 119.375, 163.485, 173.06037238907237].
2021-02-11 17:32:31,499 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010260 at: [0.0, 6.34

2021-02-11 17:32:31,843 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010608 at: [0.0, 4.210999999999999, 17.093816068269703], due to the structures at [-0.001, 0.1, 8.322, 17.094816068269704].
2021-02-11 17:32:31,853 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010609 at: [0.0, 397.3025, 436.036, 484.36199999999997, 525.331, 575.5625, 2367.7675836429958], due to the structures at [-0.001, 384.143, 410.462, 461.61, 507.114, 543.548, 607.577, 716.24, 780.737, 1216.68, 1346.946, 2367.768583642996].
2021-02-11 17:32:31,880 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF016312 at: [0.0, 186.612, 237.16290453511766], due to the structures at [-0.001, 138.879, 234.345, 237.16390453511767].
2021-02-11 17:32:31,906 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF016391 at: [0.0, 6.2459999999999996, 28.363945051202748], due to the structures at [-0.001, 0.1, 12.392, 28.36494505120275].
2021-02-11 1

### Toevoegen cross-sections

In [15]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} branches are still missing a cross section.")
print(f"{len(dfmmodel.crosssections.get_structures_without_crosssection())} structures are still missing a cross section.")

2021-02-11 17:32:36,341 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of branches without cross section is: 2385.
2021-02-11 17:32:48,711 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 863.
No parametrised crossections available for branches.
2021-02-11 17:32:48,722 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of structures without cross section is: 636.
2021-02-11 17:32:48,759 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 636.
2021-02-11 17:32:48,809 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'normgeparametriseerd' the number of structures without cross section is: 0.
863 branches are still missing a cross section.
0 structures are still missing a cross section.


### Wegschrijven model

In [16]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 144.0 * 1 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
dfmmodel.mdu_parameters["cflmax"] = 0.7

dimr_path = r"dummypath\run_dimr.bat"
dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir="dfm_model", name="boezemmodel")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r".\dfm_model\fm\boezemmodel.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven


## Importeren in de D-Hydro suite

Open het model nu in D-Hydro:
1. Open een "Empty Project"
2. In de "Home" ribbon, ga naar "Import" en selecteer "Flow Flexible Mesh Model"
3. Selecteer het bestand "boezemmodel.mdu" in ".\dfm_model\fm\boezemmodel.mdu"
4. Wacht tot het model is geimporteerd.....

Het geimporteerde model is nu zichtbaar in de D-Hydro suite
<img src="png/boezemmodel.png"/>